In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd

test_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/test.csv', encoding='iso-8859-1',na_values=[''], keep_default_na=False)

train_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/train.csv', parse_dates=['Date'], low_memory=False, na_values=[''], keep_default_na=False)

# Загрузка данных из файла store.csv
store_data = pd.read_csv('D:/Valerian/Documents/OneDrive/Python/ДопОбр Анализ данных/Практика Нетодология/sales_prediction/Команда_11/store.csv', encoding='iso-8859-1',na_values=[''], keep_default_na=False)


# Заменяем NaN на '0'
train_data = train_data.fillna("0")
test_data = test_data.fillna("0")
store_data = store_data.fillna("0")
df = pd.merge(train_data, store_data, on='Store')
#df = df.drop('Store')
print(df.columns)
print(df.info)

Удаление лишних столбцов

In [ ]:
df = df.copy()


# Определение категориальных и числовых признаков
categorical_column = ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']
num_column = ['Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 
                      'CompetitionDistance', 'CompetitionOpenSinceMonth', 
                      'Promo2', 'Promo2SinceWeek'
                      ]

date_column = ['Promo2SinceYear', 'CompetitionOpenSinceYear', "Date"]

df.info

Форматирование данных

In [ ]:
df_form = df.copy()


df_form.replace({' - ': np.nan, '\\N': np.nan, 'NaN': np.nan}, inplace=True)
df_form = df_form.dropna()

# Факторизация категориальных столбцов
encoder = OneHotEncoder(sparse_output=False)
encoded_df_list = []

for name in categorical_column:
    # Применение OneHotEncoder к целевому столбцу
    encoded_columns = encoder.fit_transform(df_form[[name]])
    # Преобразование закодированных данных в DataFrame
    encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out([name]))
    encoded_df_list.append(encoded_df)

# Объединение закодированных столбцов с исходным DataFrame (удалив при этом исходные столбцы)
df_form = df_form.drop(columns=categorical_column)
df_form = pd.concat([df_form] + encoded_df_list, axis=1)

df_form['Year'] = df_form['Date'].dt.year - 2000
df_form['Month'] = df_form['Date'].dt.month
df_form['Day'] = df_form['Date'].dt.day
del df_form['Date']
#Приведение всех числовых столбцов к типу int
for column in df_form.columns :
    df_form[column] = pd.to_numeric(df_form[column], errors='coerce')

df_form = df_form.dropna()


df_form['CompetitionOpenSinceYear'] -= 2000
df_form.loc[df_form['Promo2SinceYear'] > 2000, 'Promo2SinceYear'] -= 2000
print()

Создание тестового и тренировочного датасетов


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df_form = pd.DataFrame(df_form)

# Определяем целевую переменную
target = 'Sales'

# Разделяем признаки (X) и целевую переменную (y)
X = df_form.drop(columns=[target])
Y = df_form[target]

# Сохраняем имена колонок для последующего использования
column_names = X.columns

# Масштабируем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Создаем новый DataFrame с масштабированными данными
X_scaled_df = pd.DataFrame(X_scaled, columns=column_names)

# Разделяем данные на тренировочный и тестовый наборы
train_points, test_points, train_values, test_values = train_test_split(X_scaled_df, Y, test_size=0.2, random_state=42)

train_points = pd.DataFrame(train_points)
test_points = pd.DataFrame(test_points)
train_values = pd.DataFrame(train_values)
test_values = pd.DataFrame(test_values)


# Выводим информацию
print(train_points)
print(train_values)


Создание модели

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Определение и компиляция модели
nn_model = Sequential()
nn_model.add(Dense(17, activation='relu', input_shape=(train_points.shape[1],)))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(15, activation='relu'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_absolute_error', optimizer='adam')

# Обучение модели
results = nn_model.fit(
    train_points, train_values,
    epochs=5,
    batch_size=100,
    validation_data=(test_points, test_values)
)

# Предсказание
nn_predict = nn_model.predict(test_points).flatten()

# Удаление NaN и бесконечных значений перед расчетом mean_absolute_error
valid_indices = ~np.isnan(nn_predict) & ~np.isinf(nn_predict)
test_values_clean = test_values.values[valid_indices]
nn_predict_clean = nn_predict[valid_indices]

# Вычисление метрик
mae_nn = mean_absolute_error(test_values_clean, nn_predict_clean)
mse_nn = mean_squared_error(test_values_clean, nn_predict_clean)
rmse_nn = mse_nn ** 0.5
r2_nn = r2_score(test_values_clean, nn_predict_clean)

# Вывод значений метрик
print(f"Среднее абсолютное отклонение (MAE) на тестовых данных (NN): {mae_nn}")
print(f"Среднеквадратичная ошибка (MSE) на тестовых данных (NN): {mse_nn}")
print(f"Корень средней квадратичной ошибки (RMSE) на тестовых данных (NN): {rmse_nn}")
print(f"Коэффициент детерминации (R^2) на тестовых данных (NN): {r2_nn}")


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train_points, train_values)
y_predict = model.predict(test_points)
# Вычисление метрик
mae_rf = mean_absolute_error(test_values, y_predict)
mse_rf = mean_squared_error(test_values, y_predict)
rmse_rf = mse_rf ** 0.5
r2_rf = r2_score(test_values, y_predict)

# Вывод значений метрик
print(f"Среднее абсолютное отклонение (MAE) на тестовых данных (RandomForest): {mae_rf}")
print(f"Среднеквадратичная ошибка (MSE) на тестовых данных (RandomForest): {mse_rf}")
print(f"Корень средней квадратичной ошибки (RMSE) на тестовых данных (RandomForest): {rmse_rf}")
print(f"Коэффициент детерминации (R^2) на тестовых данных (RandomForest): {r2_rf}")

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)
xg_reg.fit(train_points, train_values)
xgb_predict = xg_reg.predict(test_points)
# Вычисление метрик
mae_rf = mean_absolute_error(y_test, xgb_predict)
mse_rf = mean_squared_error(y_test, xgb_predict)
rmse_rf = mse_rf ** 0.5
r2_rf = r2_score(y_test, xgb_predict)

# Вывод значений метрик
print(f"Среднее абсолютное отклонение (MAE) на тестовых данных (RandomForest): {mae_rf}")
print(f"Среднеквадратичная ошибка (MSE) на тестовых данных (RandomForest): {mse_rf}")
print(f"Корень средней квадратичной ошибки (RMSE) на тестовых данных (RandomForest): {rmse_rf}")
print(f"Коэффициент детерминации (R^2) на тестовых данных (RandomForest): {r2_rf}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Предположим, что ваши данные уже подготовлены и содержат колонки с лагами и скользящими средними
# Например: df содержит 'Sales_Lag_1', 'Sales_Lag_7', 'Sales_MA_7', 'Sales_MA_30' и другие признаки


    

# Разделение на признаки (X) и целевую переменную (y)
X = df_form.drop(columns=['Sales'])  # Все колонки кроме 'Sales' являются признаками
y = df_form['Sales']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Создаем модель случайного леса с оптимизированными параметрами
model = RandomForestRegressor(
    n_estimators=50,  
    max_depth=10,  # Ограничение глубины деревьев для ускорения обучения
    n_jobs=-1,  # все доступные ядра процессора
    random_state=42
)

# Обучение модели
model.fit(X_train, y_train)

# Прогнозирование
y_pred = model.predict(X_test)

# Вычисление метрик
mae_rf = mean_absolute_error(y_test, y_pred)
mse_rf = mean_squared_error(y_test, y_pred)
rmse_rf = mse_rf ** 0.5
r2_rf = r2_score(y_test, y_pred)

# Вывод значений метрик
print(f"Среднее абсолютное отклонение (MAE) на тестовых данных (RandomForest): {mae_rf}")
print(f"Среднеквадратичная ошибка (MSE) на тестовых данных (RandomForest): {mse_rf}")
print(f"Корень средней квадратичной ошибки (RMSE) на тестовых данных (RandomForest): {rmse_rf}")
print(f"Коэффициент детерминации (R^2) на тестовых данных (RandomForest): {r2_rf}")


In [ ]:
import pickle

# Сохраните модель в файл
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)